<a href="https://colab.research.google.com/github/mequanent/Bioinformatics-hw2/blob/main/hws_110761504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Clone the repository first using the following command
!npx degit https://github.com/mequanent/Bioinformatics-hw2 -f

npx: installed 1 in 2.324s
> destination directory is not empty. Using --force, continuing
> cloned mequanent/Bioinformatics-hw2#HEAD


**Homework 2**

In [4]:
#pip install Bio   # Please uncomment for the .py format

In [5]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True) # Suppress displaying the scientific notation

#To install the Bio library into my IDLE python editor I used the following two steps.
#    1. cd C:\Users\USER\AppData\Local\Programs\Python\Python39\Scripts
#    2. pip install Bio
#pip install Bio

# Read fasta and parse the seq_names and the sequences
# Reference for parsing the fasta is the original biopython library documentation
# (https://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc11 section 2.4.1)

from Bio import SeqIO
seq_name = []
sequence = []
# Remember to customize as --input from CLI args
for seq_record in SeqIO.parse("test.fasta", "fasta"):
  seq_name.append(seq_record.id)
  #sequence.append(repr(seq_record.seq))
  sequence.append(seq_record.seq)
  #print(len(seq_record))
print(sequence[1])
print(sequence[2])

def preprocess_score(score):
  file = open(score, 'r')
  lines = file.readlines()#[10:] # The first 10 lines are strings
  file.close()
  columns = []
  mut = []
  for line in lines:
    if line.startswith("#") or line.startswith(" "):
      continue
    columns.append(line[0]) # Amino acid alphabets stored for row & column indexing
    line = line[1:].strip().split() # remove the letters from left and the spaces
    mut.append(line)
  mut = np.array(mut).astype(int) # change the string numbers into type int
  print(mut.shape)
  size = len(mut) # save the size of the array for future usages to come below
  return pd.DataFrame(mut, index = columns, columns = columns)

score = "pam250.txt" # score from command line args
score = preprocess_score(score) # score made a data frame to access its values with 
                  # alphabet indices
print(score)

gop = -10
gex = -2
# A function to determine pairwise alignment score
def pairwise(x, y):
  pair_sum = 0
  x_prev = '' # x_prev and y_prev are used to store immediate previous alphabets
  y_prev = '' # so that extension gaps will be identified
  for i in range(len(x)): # Considering that both x and y have the same length
    x_prev = x_prev == '-' and x[i] == '-' # status of extension gap in the first paired sequence
    y_prev = y_prev == '-' and y[i] == '-' # status of extension gap in second paired sequence
    curr = x[i] == '-' or y[i] == '-' # status of open gap in the paired sequence
    if x_prev or y_prev: # extension gap value will be taken if holds
      pair_sum = pair_sum + gex
    elif curr: # open gap value will be taken if holds
      pair_sum = pair_sum + gop
    else: # score matrix value will be taken otherwise
      pair_sum = pair_sum + score.loc[x[i]][y[i]] 
    x_prev = x[i]
    y_prev = y[i]
  return pair_sum

pairwise(sequence[0], sequence[1])

# A function to find the total sum of pairs score.
def sumOfPairs(seq):
  sop = 0 # to store sum of pairs value
  for i in range(len(seq)-1): # i iterates as long as there is j in seq as its next sequence
    for j in range(i+1, len(seq)): # j will be the next seq index to be compared with seq[i]
      sop = sop + pairwise(seq[i], seq[j])
  return sop
print(sumOfPairs(sequence))

KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEI--WWARLNDKEGYVPRNLLGLYP
(24, 24)
   A  R  N  D   C  Q  E  G  H  I  ...  P  S  T   W   Y  V  B  Z  X  *
A  2 -2  0  0  -2  0  0  1 -1 -1  ...  1  1  1  -6  -3  0  0  0  0 -8
R -2  6  0 -1  -4  1 -1 -3  2 -2  ...  0  0 -1   2  -4 -2 -1  0 -1 -8
N  0  0  2  2  -4  1  1  0  2 -2  ...  0  1  0  -4  -2 -2  2  1  0 -8
D  0 -1  2  4  -5  2  3  1  1 -2  ... -1  0  0  -7  -4 -2  3  3 -1 -8
C -2 -4 -4 -5  12 -5 -5 -3 -3 -2  ... -3  0 -2  -8   0 -2 -4 -5 -3 -8
Q  0  1  1  2  -5  4  2 -1  3 -2  ...  0 -1 -1  -5  -4 -2  1  3 -1 -8
E  0 -1  1  3  -5  2  4  0  1 -2  ... -1  0  0  -7  -4 -2  3  3 -1 -8
G  1 -3  0  1  -3 -1  0  5 -2 -3  ...  0  1  0  -7  -5 -1  0  0 -1 -8
H -1  2  2  1  -3  3  1 -2  6 -2  ...  0 -1 -1  -3   0 -2  1  2 -1 -8
I -1 -2 -2 -2  -2 -2 -2 -3 -2  5  ... -2 -1  0  -5  -1  4 -2 -2 -1 -8
L -2 -3 -3 -4  -6 -2 -3 -4 -2  2  ... -3 -3 -2  -2  -1  2 -3 -3 -1 -8
K -1  3  1  0  -5  1  0 -2  0

**Homework 3**

In [6]:
import numpy as np
import pandas as pd

from Bio import SeqIO
seq_name = []
sequence = []
# Remember to customize as --input from CLI args
for seq_record in SeqIO.parse("test3.fasta", "fasta"):
  seq_name.append(seq_record.id)
  sequence.append(seq_record.seq)
print("----- Given Sequences ---------")
print(sequence[0], len(sequence[0]))
print(sequence[1], len(sequence[1]))

gap = -10
score = "pam250.txt" # score from command line args

def preprocess_score(score):
  file = open(score, 'r')
  lines = file.readlines()
  file.close()
  columns = []
  mut = []
  for line in lines:
    if line.startswith("#") or line.startswith(" "):
      continue
    columns.append(line[0]) # Amino acid alphabets stored for row & column indexing
    line = line[1:].strip().split() # remove the letters from left and the spaces
    mut.append(line)
  mut = np.array(mut).astype(int) # change the string numbers into type int
  return pd.DataFrame(mut, index = columns, columns = columns)

score = preprocess_score(score) # score made a data frame to access its values with 
                  # alphabet indices
#print("----- PAM Score Matrix ---------")
#print(score)

----- Given Sequences ---------
NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVN 57
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP 60


In [7]:
seq1 = sequence[0]
seq2 = sequence[1]

TOP = gap
DIAGONAL = 0
LEFT = gap
col = '-' + seq1 # This if for simplicity to avoid repeated use of (len(seq1) + 1)
row = '-' + seq2 # For the same reason

**Global Alignment Section**

In [8]:
# Filling score_m and keeping track of the direction for each cell
score_m = np.zeros((len(row), len(col)))
direction_m = np.empty((len(row), len(col)), dtype = object)
for i in range(len(col)):
  score_m[0][i] = i * gap
for j in range(len(row)):
  score_m[j][0] = j * gap

for i in range(1, len(row)):
  for j in range(1, len(col)):
    TOP = score_m[i-1,j] + gap
    DIAGONAL = score_m[i-1][j-1] + score.loc[row[i]][col[j]]
    LEFT = score_m[i,j-1] + gap
    score_m[i][j] = max(TOP, DIAGONAL, LEFT) 
    # ------ Let the direction for each cell be saved as follows 
    if score_m[i][j] == TOP:
      direction_m[i][j] = 'ins'
    elif score_m[i][j] == DIAGONAL:
      direction_m[i][j] = 'sub' 
    else:
      direction_m[i][j] = 'del'

score_m = score_m.astype(int)
F = pd.DataFrame(score_m, index = row, columns = col)
# End of filling the scoring matrix and tracking the direction

# Trace back section for global alignment
i = len(seq1)
j = len(seq2)
aln1 = ''
aln2 = ''

while(not (i == 0 and j == 0)):
  if(direction_m[j][i] == 'sub'):
    aln1 = col[i] + aln1
    aln2 = row[j] + aln2
    i = i - 1
    j = j - 1
  elif(direction_m[j][i] == 'del'):
    aln1 = col[i] + aln1
    aln2 = '-' + aln2
    i = i - 1
  else:
    aln1 = '-' + aln1
    aln2 = row[j] + aln2
    j = j - 1
# End of Trace Back for the Global Alignment
# --------------------------------------------
# Displaying Section for the Global alignment
print("+------- Given sequences ------------+")
print(seq1)
print(seq2)
print("+------- Global Alignment -----------+")
print(aln1)
print(aln2)
print("Sum of pairs (SoP) score of the alignment: ", pairwise(aln1, aln2)) 
# The SoP value verifies the correctness of our alignment
                            # if it becomes equal to the right bottom value.

+------- Given sequences ------------+
NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVN
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP
+------- Global Alignment -----------+
N-LFVALYDFVASGDNTLSITKGEKLRVLGY-NHNG-EWCEAQTKNGQGWVPSNYITPVN
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP
Sum of pairs (SoP) score of the alignment:  38


**Local Alignment**

In [15]:
score_local = np.zeros((len(row), len(col))).astype(int)
bestScore = 0
bestScoreCell = (0, 0)
direction_l = np.zeros((len(row), len(col)), dtype = object)

for i in range(1, len(row)):
  for j in range(1, len(col)):
    TOP = score_local[i-1,j] + gap
    DIAGONAL = score_local[i-1][j-1] + score.loc[row[i]][col[j]]
    LEFT = score_local[i,j-1] + gap
    score_local[i][j] = max(TOP, DIAGONAL, LEFT, 0)

    if score_local[i][j] >= bestScore:
      bestScore = score_local[i][j]
      bestScoreCell = (i, j)
    
    if score_local[i][j] == TOP:
      direction_l[i][j] = 'ins'
    elif score_local[i][j] == DIAGONAL:
      direction_l[i][j] = 'sub' 
    else:
      direction_l[i][j] = 'del'

local_df = pd.DataFrame(score_local, index = row, columns = col)
local_df
direction_l

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 'sub', 'del', ..., 'del', 'del', 'sub'],
       [0, 'sub', 'del', ..., 'sub', 'del', 'sub'],
       ...,
       [0, 'del', 'sub', ..., 'sub', 'sub', 'del'],
       [0, 'del', 'del', ..., 'ins', 'sub', 'sub'],
       [0, 'sub', 'del', ..., 'sub', 'sub', 'sub']], dtype=object)

In [50]:
print(bestScore)
print(bestScoreCell)

52
(57, 54)


In [10]:
# Trace back the local alignment
i = bestScoreCell[0]
j = bestScoreCell[1]
print('Best from row: ', row[i])
print('Best from col: ', col[i])
al1 = ''
al2 = ''

while i > 0 and j > 0:
  TOP = score_local[i-1][j]
  DIAGONAL = score_local[i-1][j-1]
  LEFT = score_local[i][j-1]

  if min(TOP, DIAGONAL, LEFT) == DIAGONAL:
    #break
    al1 = '-' + al1
    al2 = '-' + al2
    i = i - 1
    j = j - 1
  else:
    al1 = col[j] + al1
    al2 = row[i] + al2
    i = i - 1
    j = j - 1

print('+------------------ Given Sequences ------------------+')
print(seq1)
#print(seq2)
#print('+------------------ Local Alignment ------------------+')
print(al1)
print(al2)
print(seq2)
print(pairwise(al1, al2))

Best from row:  G
Best from col:  N
+------------------ Given Sequences ------------------+
NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVN
----A--DFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYIT
----W--EPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLG
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP
-21


In [13]:
i = bestScoreCell[0]
j = bestScoreCell[1]
print('Best from row: ', row[i])
print('Best from col: ', col[j])

Best from row:  G
Best from col:  T


In [18]:
# Trace back the local alignment
i = bestScoreCell[0]
j = bestScoreCell[1]
print('Best from row: ', row[i])
print('Best from col: ', col[j])
print('Best score: ', bestScore)
#print(len(col))
al1 = ''
al2 = ''

while((i > 0 and j > 0) and score_local[i][j]> 0):
  if(direction_l[i][j] == 'sub'):
    al1 = col[i] + al1
    al2 = row[j] + al2
    i = i - 1
    j = j - 1
  elif(direction_l[i][j] == 'del'):
    al1 = col[i] + al1
    al2 = '-' + al2
    i = i - 1
    j = j - 1
  else:
    al1 = '-' + al1
    al2 = row[j] + al2
    i = i - 1
    j = j - 1

print('+------------------ Given Sequences ------------------+')
print(seq1)
#print(seq2)
#print('+------------------ Local Alignment ------------------+')
print(al1)
print(al2)
print(seq2)

Best from row:  G
Best from col:  T
Best score:  52
+------------------ Given Sequences ------------------+
NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVN
VASGDNTLSIT-------LG--HNGE--EAQTKNGQGWVPSNYITPVN
LWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRN
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP


In [32]:
local_df

,-,N,L,F,V,A,L,Y,D,F,...,V,P,S,N,Y,I,T,P,V,N
-,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
K,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
G,0,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
V,0,0,2,0,4,0,3,0,0,0,...,4,0,0,0,0,4,0,0,4,0
I,0,0,2,3,4,3,2,2,0,1,...,4,2,0,0,0,5,4,0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L,0,0,6,10,4,0,6,5,0,5,...,4,11,20,31,40,52,42,32,25,15
G,0,0,0,1,9,5,0,1,6,0,...,0,4,12,21,30,42,52,42,32,25
L,0,0,6,2,3,7,11,1,0,8,...,2,0,2,11,20,32,42,49,44,34
Y,0,0,0,13,3,0,6,21,11,7,...,1,0,0,1,21,22,32,39,47,42


In [39]:
bestScoreCell[1]

54